变量重命名的主要目的之一是方便使用滑动平均值。

每一个变量的滑动平均值是通过影子变量维护的。如果在加载模型时直接将影子变量映射到变量自身，那么在使用训练好的模型时就不需要再调用函数来获取变量的滑动平均值了。

In [1]:
import tensorflow as tf

#### 1. 使用滑动平均。

In [2]:
v = tf.Variable(0, dtype=tf.float32, name="v")
# 在没有声明滑动平均模型时，只有一个变量v
for variables in tf.global_variables(): 
    print("variables.name:", variables.name)
    
ema = tf.train.ExponentialMovingAverage(0.99)
maintain_averages_op = ema.apply(tf.global_variables())
# 申明滑动平均模型之后，会自动生成一个影子变量，所以这里有两个变量
for variables in tf.global_variables(): 
    print("variables.name:", variables.name)

variables.name: v:0
variables.name: v:0
variables.name: v/ExponentialMovingAverage:0


#### 2. 保存滑动平均模型。

In [3]:
saver = tf.train.Saver()
with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    
    sess.run(tf.assign(v, 10))
    sess.run(maintain_averages_op)
    # 保存的时候会将v:0  v/ExponentialMovingAverage:0这两个变量都存下来。
    saver.save(sess, "Saved_model/model2.ckpt")
    print(sess.run([v, ema.average(v)]))

[10.0, 0.099999905]


#### 3. 加载滑动平均模型。

In [4]:
v = tf.Variable(0, dtype=tf.float32, name="v")

# 通过变量重命名将原来变量v的滑动平均值直接赋值给v。
saver = tf.train.Saver({"v/ExponentialMovingAverage": v})
with tf.Session() as sess:
    saver.restore(sess, "Saved_model/model2.ckpt")
    print(sess.run(v))

INFO:tensorflow:Restoring parameters from Saved_model/model2.ckpt
0.099999905
